In [1]:
from src.gaussian_simulation import *
from src.estimators2 import *
from src.sgd import *
import numpy as np
from numpy.linalg import norm 
import matplotlib.pylab as plt

On reprend le cadre précédent. On génére un n-échantillon de taille 1000, de loi $p_{\theta}(\mathbf{x}|z)=\mathcal{N}(z, I_{20})$. Pour ce faire, on simule préalablement un échantillon de loi $p_{\theta}(z)$.

In [24]:
np.random.seed(123)

theta=simulate_gaussian_vector(mu=np.array([0]*20), sigma=np.identity(20))
A=np.identity(20)*0.5
b=theta/2
echantillon_x=generer_nech_gaussien_x_sachant_z(n=1024, mu_z=theta, sigma_z=np.identity(20),sigma_x=np.identity(20))
print("Vraie valeur de theta: {}".format(np.around(theta, 2)))

Vraie valeur de theta: [-1.09  1.    0.28 -1.51 -0.58  1.65 -2.43 -0.43  1.27 -0.87 -0.68 -0.09
  1.49 -0.64 -0.44 -0.43  2.21  2.19  1.    0.39]


La log-vraisemblance de l'échantillon (avec le facteur $-\frac{1}{n}$) est $l_n(\theta)=\frac{1}{n}\sum_{i=1}^n \lVert X_i - \theta \rVert^2 = \frac{1}{n}\sum_{i=1}^n l_i(\theta) $. 
On cherche $\hat{\theta} \in argmin \; l_n(\theta)$

# Une première estimation de $\theta$

On utilise les estimateurs du gradients de la log-vraisemblance pour estimer $\theta$ à l'aide de la descente de gradient stochastique. On fixe les paramètres de coût computationnel au mimimum pour faciliter la comparaison avec l'estimateur de $\theta$ par SGD usuelle ($k=6$ pour l'estimateur IAWE et à $l=0$ pour les autres). Dans la descente de gradient stochastique usuelle, on estime le gradient de la log vraisemblance par $\nabla_{\theta} l_i(\theta)= -2(X_i-\theta)$. Les hyper-paramètres sont fixés de la façon suivante:
* Nombre d'itérations: 100
* Learning rate: 0,001
* Comme $\theta$ est proche de $0_{\mathbb{R}^{20}}$, on fixe $\theta_0$ à $0_{\mathbb{R}^{20}}$.

In [23]:
np.random.seed(123)
theta_SGD=SGD(theta_init=np.random.uniform(low=-5, high=5, size=20), learn_rate=0.01, n_iter=1536, echantillon=echantillon_x)
theta_iawe=SGD_IAWE(theta_init=np.random.uniform(low=-5, high=5, size=20), learn_rate=0.01, n_iter=1536, A=A, b=b, echantillon=echantillon_x, k=6)
theta_SUMO=SGD_SUMO(theta_init=np.random.uniform(low=-5, high=5, size=20), learn_rate=0.001, n_iter=1536, A=A, b=b, echantillon=echantillon_x, l=0)
theta_RR=SGD_RR(theta_init=np.random.uniform(low=-5, high=5, size=20), learn_rate=0.001, n_iter=1536, A=A, b=b, echantillon=echantillon_x, l=0)
theta_SS=SGD_SS(theta_init=np.random.uniform(low=-5, high=5, size=20), learn_rate=0.001, n_iter=1536, A=A, b=b, echantillon=echantillon_x, l=0)

print("Vraie valeur de theta: {}".format(np.around(theta, 2)))
print("Estimation de theta par descente de gradient stochastique usuelle: {}".format(np.around(theta_SGD, 2)))
print("Estimation de theta par SGD IWAE: {}".format(np.around(theta_iawe, 2)))
print("Estimation de theta par SGD SUMO: {}".format(np.around(theta_SUMO, 2)))
print("Estimation de theta par SGD RR: {}".format(np.around(theta_RR, 2)))
print("Estimation de theta par SGD SS: {}".format(np.around(theta_SS, 2)))

Vraie valeur de theta: [-1.09  1.    0.28 -1.51 -0.58  1.65 -2.43 -0.43  1.27 -0.87 -0.68 -0.09
  1.49 -0.64 -0.44 -0.43  2.21  2.19  1.    0.39]
Estimation de theta par descente de gradient stochastique usuelle: [-0.97  1.01  0.13 -1.44 -0.68  1.57 -2.61 -0.51  1.48 -0.55 -0.72  0.09
  1.27 -0.62 -0.28 -0.22  2.27  2.16  0.94  0.22]
Estimation de theta par SGD IWAE: [ -0.53 -10.33   6.33   9.51  -7.16  -8.    12.77   5.77  -2.35  -7.42
  13.36   6.89 -13.85  -1.1   -9.9    8.03   2.65   4.65 -12.33 -10.61]
Estimation de theta par SGD SUMO: [  6.36   2.69 -13.24  -2.09   4.42  -5.11  11.52  -6.28   5.41  12.08
   2.79   0.41 -13.41   0.7   12.23  -5.28   0.45 -15.27  -1.52  -8.84]
Estimation de theta par SGD RR: [-22.39 -11.17   1.13  -6.76   5.86   0.37  20.78   8.55 -11.19   2.82
  -5.43  11.27  -1.34  16.11  -6.06  -1.65   4.62  -0.38  -5.39  -4.37]
Estimation de theta par SGD SS: [-11.3    6.51  -9.65   9.75   7.31  -9.99  -0.52  -8.1  -10.08   6.6
  -5.3   -7.66  -2.84   5.54  -7.

# 1ère procédure de Monte Carlo: comparaison des cinq estimateurs du gradient de la log-vraisemblance pour la SGD

On fixe les paramètres computationnels des estimateurs IWAE, SUMO, ML-SS et ML-RR de sorte que leur coût computationnel espéré soit minimal.

La procédure de Monte Carlo utilisée est la suivante:

Pour m allant de 1 à M:
* Générer $(z_1, ..., z_n) \sim p_{\theta}(z)$
* Générer $(x_1, ..., x_n) \sim p_{\theta}(x|z)$
* Calculer $\widehat{\theta}_m^j$: estimateur de $\theta$ calculé par descente de gradient stochastique, selon l'estimateur $j$ du gradient considéré.

Fin pour

Retourner $Biais_{M,j}$, $Var_{M,j}$ pour chaque estimateur $j$ considéré.

In [14]:
def procedure_MC_theta(M, n=1000, mu_z=theta, sigma_z=np.identity(20),sigma_x=np.identity(20)):

    estimations_SGD_M=np.array([])
    estimations_IWAE_M=np.array([])
    estimations_SUMO_M=np.array([])
    estimations_SS_M=np.array([])
    estimations_RR_M=np.array([])

    m=1
    while m<=M:
        echantillon_x=generer_nech_gaussien_x_sachant_z(n=n, mu_z=mu_z, sigma_z=sigma_z,sigma_x=sigma_x)
        theta_SGD=SGD(theta_init=np.array([0]*20), learn_rate=0.001, echantillon=echantillon_x, n_iter=100)
        theta_IAWE=SGD_IAWE(theta_init=np.array([0]*20), learn_rate=0.001, n_iter=100, A=A, b=b, echantillon=echantillon_x, k=6)
        theta_SUMO=SGD_SUMO(theta_init=np.array([0]*20), learn_rate=0.001, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=0)
        theta_RR=SGD_RR(theta_init=np.array([0]*20), learn_rate=0.001, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=0)
        theta_SS=SGD_SS(theta_init=np.array([0]*20), learn_rate=0.001, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=0)

        if m==1:
            estimations_SGD_M=np.append(estimations_SGD_M, theta_SGD)
            estimations_IWAE_M= np.append(estimations_IWAE_M, theta_IAWE)
            estimations_SUMO_M=np.append(estimations_SUMO_M, theta_SUMO)
            estimations_RR_M=np.append(estimations_RR_M, theta_RR)
            estimations_SS_M=np.append(estimations_SS_M, theta_SS)
                
        else:
            estimations_SGD_M=np.vstack((estimations_SGD_M, theta_SGD))
            estimations_IWAE_M= np.vstack((estimations_IWAE_M, theta_IAWE))
            estimations_SUMO_M=np.vstack((estimations_SUMO_M, theta_SUMO))
            estimations_RR_M=np.vstack((estimations_RR_M, theta_RR))
            estimations_SS_M=np.vstack((estimations_SS_M, theta_SS))
        m+=1

    biais_SGD_M=np.mean(estimations_SGD_M, axis=0)-theta
    biais_IWAE_M=np.mean(estimations_IWAE_M, axis=0)-theta
    biais_IWAE_M=np.mean(estimations_IWAE_M, axis=0)-theta
    biais_SUMO_M=np.mean(estimations_SUMO_M, axis=0)-theta
    biais_SS_M=np.mean(estimations_SS_M, axis=0)-theta
    biais_RR_M=np.mean(estimations_RR_M, axis=0)-theta

    squared_biais_SGD_M=norm(biais_SGD_M)**2
    squared_biais_IWAE_M=norm(biais_IWAE_M)**2
    squared_biais_SUMO_M=norm(biais_SUMO_M)**2
    squared_biais_SS_M=norm(biais_SS_M)**2
    squared_biais_RR_M=norm(biais_RR_M)**2

    var_SGD_M=np.mean(norm(estimations_SGD_M-np.mean(estimations_IWAE_M, axis=0), axis=1)**2)
    var_IWAE_M=np.mean(norm(estimations_IWAE_M-np.mean(estimations_IWAE_M, axis=0), axis=1)**2)
    var_SUMO_M=np.mean(norm(estimations_SUMO_M-np.mean(estimations_SUMO_M, axis=0), axis=1)**2)
    var_SS_M=np.mean(norm(estimations_SS_M-np.mean(estimations_SS_M, axis=0), axis=1)**2)
    var_RR_M=np.mean(norm(estimations_RR_M-np.mean(estimations_RR_M, axis=0), axis=1)**2)

    return squared_biais_SGD_M, squared_biais_IWAE_M, squared_biais_SUMO_M, squared_biais_SS_M, squared_biais_RR_M, var_SGD_M, var_IWAE_M, var_SUMO_M, var_SS_M, var_RR_M

In [15]:
np.random.seed(123)

squared_biais_SGD_M, squared_biais_IWAE_M, squared_biais_SUMO_M, squared_biais_SS_M, squared_biais_RR_M, var_SGD_M, var_IWAE_M, var_SUMO_M, var_SS_M, var_RR_M=procedure_MC_theta(M=1000, n=1000,
                                                                                                                                                                                        mu_z=theta,
                                                                                                                                                                                        sigma_z=np.identity(20),
                                                                                                                                                                                        sigma_x=np.identity(20))

In [16]:
print("Biais au carré de l'estimateur de theta par SGD usuelle: {}".format(round(squared_biais_SGD_M,2)))
print("Biais au carré de l'estimateur de theta par SGD IWAE: {}".format(round(squared_biais_IWAE_M,2)))
print("Biais au carré de l'estimateur de theta par SGD SUMO: {}".format(round(squared_biais_SUMO_M,2)))
print("Biais au carré de l'estimateur de theta par SGD ML-SS: {}".format(round(squared_biais_SS_M,2)))
print("Biais au carré de l'estimateur de theta par SGD ML-RR: {}".format(round(squared_biais_RR_M,2)))

print("Variance de l'estimateur de theta par SGD usuelle: {}".format(round(var_SGD_M,2)))
print("Variance de l'estimateur de theta par SGD IWAE: {}".format(round(var_IWAE_M,2)))
print("Variance de l'estimateur de theta par SGD SUMO: {}".format(round(var_SUMO_M,2)))
print("Variance de l'estimateur de theta par SGD ML-SS: {}".format(round(var_SS_M,2)))
print("Variance de l'estimateur de theta par SGD ML-RR: {}".format(round(var_RR_M,2)))

Biais au carré de l'estimateur de theta par SGD usuelle: 20.3
Biais au carré de l'estimateur de theta par SGD IWAE: 35.79
Biais au carré de l'estimateur de theta par SGD SUMO: 40.0
Biais au carré de l'estimateur de theta par SGD ML-SS: nan
Biais au carré de l'estimateur de theta par SGD ML-RR: nan
Variance de l'estimateur de theta par SGD usuelle: 2.2
Variance de l'estimateur de theta par SGD IWAE: 0.0
Variance de l'estimateur de theta par SGD SUMO: 0.01
Variance de l'estimateur de theta par SGD ML-SS: nan
Variance de l'estimateur de theta par SGD ML-RR: nan


# 2e procédure de Monte Carlo: comparaison des estimateurs du gradient paramétrés par un paramètre de complexité computationnelle

Comme l'estimateur de $\theta$  par SDG usuelle ne présente pas de paramètre de coût computationnel $k$ ou $l$, nous comparons uniqement les performances de IWAE, SUMO, ML-SS et ML-RR entre-eux, en fonction du coût computationnel espéré. Pour ce faire, nous répliquons la procédure de Monte-Carlo présentée dans la partie traitant de l'estimation de la log-vraisemblance et de son gradient.

In [17]:
def procedure_MC_theta(M, L, theta, A, b, n):
    biais_IWAE_M={}
    biais_SUMO_M={}
    biais_SS_M={}
    biais_RR_M={}

    var_IWAE_M={}
    var_SUMO_M={}
    var_SS_M={}
    var_RR_M={}

    l=0
    while l<=L:
        m=1
        estimations_IWAE_M_l=np.array([])
        estimations_SUMO_M_l=np.array([])
        estimations_SS_M_l=np.array([])
        estimations_RR_M_l=np.array([])


        while m<=M:
            echantillon_x=generer_nech_gaussien_x_sachant_z(n=n, mu_z=theta, sigma_z=np.identity(20),sigma_x=np.identity(20))

            theta_IAWE=SGD_IAWE(theta_init=np.array([0]*20), learn_rate=0.001, n_iter=100, A=A, b=b, echantillon=echantillon_x, k=6*2**l)
            theta_SUMO=SGD_SUMO(theta_init=np.array([0]*20), learn_rate=0.001, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=l*38)
            theta_RR=SGD_RR(theta_init=np.array([0]*20), learn_rate=0.001, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=l)
            theta_SS=SGD_SS(theta_init=np.array([0]*20), learn_rate=0.001, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=l)

            if m==1:
                estimations_IWAE_M_l= np.append(estimations_IWAE_M_l, theta_IAWE)
                estimations_SUMO_M_l=np.append(estimations_SUMO_M_l, theta_SUMO)
                estimations_RR_M_l=np.append(estimations_RR_M_l, theta_RR)
                estimations_SS_M_l=np.append(estimations_SS_M_l, theta_SS)
            
            else:
                estimations_IWAE_M_l= np.vstack((estimations_IWAE_M_l, theta_IAWE))
                estimations_SUMO_M_l=np.vstack((estimations_SUMO_M_l, theta_SUMO))
                estimations_RR_M_l=np.vstack((estimations_RR_M_l, theta_RR))
                estimations_SS_M_l=np.vstack((estimations_SS_M_l, theta_SS))
            m+=1

        biais_IWAE_M_l=np.mean(estimations_IWAE_M_l, axis=0)-theta
        biais_SUMO_M_l=np.mean(estimations_SUMO_M_l, axis=0)-theta
        biais_SS_M_l=np.mean(estimations_SS_M_l, axis=0)-theta
        biais_RR_M_l=np.mean(estimations_RR_M_l, axis=0)-theta

        squared_biais_IWAE_M_l=norm(biais_IWAE_M_l)**2
        squared_biais_SUMO_M_l=norm(biais_SUMO_M_l)**2
        squared_biais_SS_M_l=norm(biais_SS_M_l)**2
        squared_biais_RR_M_l=norm(biais_RR_M_l)**2

        var_IWAE_M_l=np.mean(norm(estimations_IWAE_M_l-np.mean(estimations_IWAE_M_l, axis=0), axis=1)**2)
        var_SUMO_M_l=np.mean(norm(estimations_SUMO_M_l-np.mean(estimations_SUMO_M_l, axis=0), axis=1)**2)
        var_SS_M_l=np.mean(norm(estimations_SS_M_l-np.mean(estimations_SS_M_l, axis=0), axis=1)**2)
        var_RR_M_l=np.mean(norm(estimations_RR_M_l-np.mean(estimations_RR_M_l, axis=0), axis=1)**2)
        
        biais_IWAE_M[l]=squared_biais_IWAE_M_l
        biais_SUMO_M[l]=squared_biais_SUMO_M_l
        biais_SS_M[l]=squared_biais_SS_M_l
        biais_RR_M[l]=squared_biais_RR_M_l
        
        var_IWAE_M[l]=var_IWAE_M_l
        var_SUMO_M[l]=var_SUMO_M_l
        var_SS_M[l]=var_SS_M_l
        var_RR_M[l]=var_RR_M_l

        l+=1

    return biais_IWAE_M, biais_SUMO_M, biais_SS_M, biais_RR_M, var_SGD_M,var_IWAE_M, var_SUMO_M, var_SS_M, var_RR_M

In [18]:
np.random.seed(123)

biais_SGD_M_theta, biais_IWAE_M_theta, biais_SUMO_M_theta, biais_SS_M_theta, biais_RR_M_theta, var_SGD_M_theta, var_IWAE_M_theta, var_SUMO_M_theta, var_SS_M_theta, var_RR_M_theta = procedure_MC_theta(M=1000, 
                                                                                                                                                                                                        L=6,
                                                                                                                                                                                                        theta=theta,
                                                                                                                                                                                                        A=A, 
                                                                                                                                                                                                        b=b,
                                                                                                                                                                                                        n=1000) 
                                                                                                                            

KeyboardInterrupt: 

In [ ]:
sorted_biais_IWAE_M_theta = sorted(biais_IWAE_M_theta.items())
sorted_biais_SUMO_M_theta = sorted(biais_SUMO_M_theta.items())
sorted_biais_SS_M_theta = sorted(biais_SS_M_theta.items())
sorted_biais_RR_M_theta = sorted(biais_RR_M_theta.items())

sorted_var_IWAE_M_theta = sorted(var_IWAE_M_theta.items())
sorted_var_SUMO_M_theta = sorted(var_SUMO_M_theta.items())
sorted_var_SS_M_theta = sorted(var_SS_M_theta.items())
sorted_var_RR_M_theta = sorted(var_RR_M_theta.items())

l, biais_IAWE_theta = zip(*sorted_biais_IWAE_M_theta)
l, biais_SUMO_theta = zip(*sorted_biais_SUMO_M_theta)
l, biais_SS_theta = zip(*sorted_biais_SS_M_theta)
l, biais_RR_theta = zip(*sorted_biais_RR_M_theta)

l, var_IAWE_theta = zip(*sorted_var_IWAE_M_theta) 
l, var_SUMO_theta = zip(*sorted_var_SUMO_M_theta) 
l, var_SS_theta = zip(*sorted_var_SS_M_theta) 
l, var_RR_theta = zip(*sorted_var_RR_M_theta) 

fig, axs = plt.subplots(2,1, figsize=(10, 10))

axs[0].plot(l, biais_IAWE_theta, 'b', label="Biais au carré de l'estimateur IWAE", color="blue")
axs[0].plot(l, biais_SUMO_theta, 'b', label="Biais au carré de l'estimateur SUMO", color="red")
axs[0].plot(l, biais_SS_theta, 'b', label="Biais au carré de l'estimateur ML-SS", color="orange")
axs[0].plot(l, biais_RR_theta, 'b', label="Biais au carré de l'estimateur ML-RR", color="green")

axs[0].legend()

fig.suptitle('Biais au carré et variance des estimateurs de theta, en fonction du coût computationel', fontsize=16)

axs[1].plot(l, var_IAWE_theta, 'b', label="Variance de l'estimateur IAWE", color="blue")
axs[1].plot(l, var_SUMO_theta, 'b', label="Variance de l'estimateur SUMO", color="red")
axs[1].plot(l, var_SS_theta, 'b', label="Variance de l'estimateur ML-SS", color="orange")
axs[1].plot(l, var_RR_theta, 'b', label="Variance de l'estimateur ML-RR", color="green")


axs[1].legend()

plt.yscale("log")
plt.show()

NameError: name 'biais_SDG_M_theta' is not defined